In [1]:
# %load_ext cuml.accel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os  # 导入os库来处理文件和目录
import glob # 导入glob库来查找文件

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from scipy.signal import welch
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:


# def load_all_segments(root_dir, eeg_suffix='EEG_aligned.npy', sound_suffix='Sound_aligned.npy', root_suffix='feature_normalized'):

#     eeg_segments_list = []
#     sound_segments_list = []

#     subject_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

#     if not subject_folders:
#         print(f"错误: 在 '{root_dir}' 中没有找到任何被试者文件夹。")
#         return None, None

#     print(f"找到了 {len(subject_folders)} 个被试者...")

#     for subject in subject_folders:
#         subject_path = os.path.join(root_dir, subject, root_suffix)

#         eeg_files = glob.glob(os.path.join(subject_path, f'*{eeg_suffix}'))

#         if not eeg_files:
#             continue

#         for eeg_file_path in eeg_files:
#             base_name = os.path.basename(eeg_file_path).replace(eeg_suffix, '')
#             sound_file_path = os.path.join(subject_path, base_name + sound_suffix)

#             if os.path.exists(sound_file_path):
#                 try:
#                     eeg_segment = np.load(eeg_file_path)
#                     sound_segment = np.load(sound_file_path)


#                     if eeg_segment.shape[0] == sound_segment.shape[0]:
#                         eeg_segments_list.append(eeg_segment)
#                         sound_segments_list.append(sound_segment)
#                 except Exception as e:
#                     print(f"加载文件 {os.path.basename(eeg_file_path)} 时出错: {e}")

#     if not eeg_segments_list:
#         print("错误: 未能加载任何有效的数据段。")
#         return None, None

#     # --- 最终修改在这里 ---
#     # 我们不再使用 np.vstack
#     # 直接返回包含所有数据段的列表
#     print(f"\n加载完成！总共加载了 {len(eeg_segments_list)} 个数据段。")
#     return eeg_segments_list, sound_segments_list

def load_segments_with_subject_ids(root_dir, eeg_suffix='EEG_aligned.npy', sound_suffix='Sound_aligned.npy', root_suffix='feature_normalized'):
    eeg_segments_list = []
    sound_segments_list = []
    subject_ids_list = [] # 新增一个列表来存储ID

    print(f"开始从根目录 '{root_dir}' 加载数据段及被试者ID...")

    subject_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

    for subject in tqdm(subject_folders, desc="处理被试者"):
        subject_path = os.path.join(root_dir, subject, root_suffix)
        eeg_files = glob.glob(os.path.join(subject_path, f'*{eeg_suffix}'))

        for eeg_file_path in eeg_files:
            base_name = os.path.basename(eeg_file_path).replace(eeg_suffix, '')
            sound_file_path = os.path.join(subject_path, base_name + sound_suffix)

            if os.path.exists(sound_file_path):
                try:
                    eeg_segment = np.load(eeg_file_path)
                    sound_segment = np.load(sound_file_path)

                    if eeg_segment.shape[0] == sound_segment.shape[0]:
                        eeg_segments_list.append(eeg_segment)
                        sound_segments_list.append(sound_segment)
                        subject_ids_list.append(subject) # 关键：记录下当前数据段属于哪个被试者
                except Exception as e:
                    print(f"加载文件 {os.path.basename(eeg_file_path)} 时出错: {e}")

    print(f"\n加载完成！总共加载了 {len(eeg_segments_list)} 个数据段。")
    return eeg_segments_list, sound_segments_list, subject_ids_list


In [4]:
ROOT_DATA_DIR = r'/content/drive/MyDrive/data'


# 假设你的原始数据段文件名后缀是 '_eeg_segment.npy' 和 '_sound_segment.npy'
# 请务必根据你的实际情况修改！

# 调用新函数
eeg_segments, sound_segments, subject_ids_list = load_segments_with_subject_ids(ROOT_DATA_DIR)

# 检查一下我们的“档案箱”里有什么
if eeg_segments is not None:
    print("\n--- 加载结果检查 ---")
    print(f"eeg_segments 是一个列表，长度为: {len(eeg_segments)}")
    print(f"列表中的第一个EEG数据段的形状是: {eeg_segments[0].shape}")
    print(f"列表中的第二个EEG数据段的形状是: {eeg_segments[1].shape}")

    print(f"\nsound_segments 也是一个列表，长度为: {len(sound_segments)}")
    print(f"列表中的第一个声音数据段的形状是: {sound_segments[0].shape}")

开始从根目录 '/content/drive/MyDrive/data' 加载数据段及被试者ID...


处理被试者: 100%|██████████| 20/20 [01:01<00:00,  3.08s/it]


加载完成！总共加载了 1794 个数据段。

--- 加载结果检查 ---
eeg_segments 是一个列表，长度为: 1794
列表中的第一个EEG数据段的形状是: (364, 32)
列表中的第二个EEG数据段的形状是: (364, 32)

sound_segments 也是一个列表，长度为: 1794
列表中的第一个声音数据段的形状是: (364, 54)


In [5]:
SAMPLING_RATE = 256
BANDS = {
    'delta': [1, 4],    # 1-4 Hz
    'theta': [4, 8],    # 4-8 Hz
    'alpha': [8, 13],   # 8-13 Hz
    'beta': [13, 30],   # 13-30 Hz
    'gamma': [30, 50]   # 30-50 Hz
}

In [6]:
def extract_features_from_segments(eeg_segment, sound_segment, sampling_rate, bands):
    """
    对单个数据段进行特征提取
    eeg_segment: 形状为 (n_samples, 32) 的EEG数据段
    sound_segment: 形状为 (n_samples, 54) 的声音数据段
    """

    # --- 1. 提取EEG的频域特征 (生成 X 的一行) ---
    psd_features = []
    # 遍历32个EEG通道
    for channel_index in range(eeg_segment.shape[1]):
        # 使用welch方法计算功率谱密度(PSD)
        # freqs: 频率点, psd: 对应频率点的能量
        freqs, psd = welch(eeg_segment[:, channel_index], fs=sampling_rate, nperseg=sampling_rate) # 用1秒的窗口计算

        # 计算每个预定义频带的平均能量
        for band, freq_range in bands.items():
            # 找到落在当前频带内的频率点的索引
            band_indices = np.where((freqs >= freq_range[0]) & (freqs <= freq_range[1]))[0]
            # 计算这些频率点上的平均能量，如果没找到则为0
            band_power = np.mean(psd[band_indices]) if len(band_indices) > 0 else 0
            psd_features.append(band_power)

    # --- 2. 计算声音特征的均值作为目标 (生成 Y 的一行) ---
    sound_features_mean = np.mean(sound_segment, axis=0)

    return np.array(psd_features), sound_features_mean

# --- 整合流水线: 遍历所有数据段并提取特征 ---
X_new = []
Y_new = []

In [7]:
for i in tqdm(range(len(eeg_segments)), desc="提取特征中"):
    eeg_seg = eeg_segments[i]
    sound_seg = sound_segments[i]

    # 调用核心函数
    x_feat, y_mean = extract_features_from_segments(eeg_seg, sound_seg, SAMPLING_RATE, BANDS)

    X_new.append(x_feat)
    Y_new.append(y_mean)

提取特征中: 100%|██████████| 1794/1794 [00:37<00:00, 47.31it/s]


In [8]:
X_df = pd.DataFrame(X_new)
X_df['subject'] = subject_ids_list

# 创建StandardScaler实例
scaler = StandardScaler()

# 创建一个空列表来收集处理好的数据块
normalized_blocks = []

# 使用 groupby 循环，为每个被试者独立进行标准化
for subject_name, subject_data in tqdm(X_df.groupby('subject'), desc="按被试者归一化"):
    # 分离出特征列
    features = subject_data.drop('subject', axis=1)
    # 对当前被试者的数据块进行 fit_transform
    normalized_features = scaler.fit_transform(features)
    # 将处理好的数据块存入列表
    normalized_blocks.append(pd.DataFrame(normalized_features, index=features.index))

# 将所有标准化的数据块重新合并
X_final_df = pd.concat(normalized_blocks).sort_index()
X_final = X_final_df.to_numpy()

# Y 通常进行全局处理，因为它是我们的目标标准
Y_final = Y_new

按被试者归一化: 100%|██████████| 15/15 [00:00<00:00, 230.34it/s]


In [9]:
# 将结果列表转换为Numpy数组
X_final = np.array(X_final)
Y_final = np.array(Y_final)

print("\n--- 特征提取完成！---")
print("最终数据集已生成。")
print(f"新特征矩阵 X_new 的形状: {X_final.shape}")
print(f"新目标矩阵 Y_new 的形状: {Y_final.shape}")


--- 特征提取完成！---
最终数据集已生成。
新特征矩阵 X_new 的形状: (1794, 160)
新目标矩阵 Y_new 的形状: (1794, 54)


In [10]:

X_train, X_test, Y_train, Y_test = train_test_split(
    X_final, Y_final, test_size=0.2, random_state=42
)
print("新数据集划分完成。")
print(f"X_train 形状: {X_train.shape}, Y_train 形状: {Y_train.shape}")


# =============================================================================
# 步骤 2: 对 Y (目标) 进行提纯
# 即使是Y_new，原始的54个特征维度依然存在冗余，所以我们重复之前的成功经验
# =============================================================================
print("\n正在对目标数据 Y 进行“填充-标准化-PCA”提纯...")




新数据集划分完成。
X_train 形状: (1435, 160), Y_train 形状: (1435, 54)

正在对目标数据 Y 进行“填充-标准化-PCA”提纯...


In [11]:
# 创建并配置Y的处理工具
imputer_y = SimpleImputer(strategy='mean')
scaler_y = StandardScaler()
# 让PCA自动选择能保留95%信息的主成分数量
pca_y = PCA(n_components=0.95)

# 对训练集进行fit_transform
Y_train_imputed = imputer_y.fit_transform(Y_train)
Y_train_scaled = scaler_y.fit_transform(Y_train_imputed)
Y_train_pca = pca_y.fit_transform(Y_train_scaled)

# 对测试集只进行transform
Y_test_imputed = imputer_y.transform(Y_test)
Y_test_scaled = scaler_y.transform(Y_test_imputed)
Y_test_pca = pca_y.transform(Y_test_imputed)

print(f"Y 已被成功提纯至 {pca_y.n_components_} 个主成分。")


Y 已被成功提纯至 25 个主成分。


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [52]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [52]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [12]:
print("\n正在对输入数据 X (频域特征) 进行标准化...")
scaler_x = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)


print("\n开始训练最终的随机森林模型...")


正在对输入数据 X (频域特征) 进行标准化...

开始训练最终的随机森林模型...


In [13]:
final_rf_model = RandomForestRegressor(
    n_estimators=200,       # 更多的树
    random_state=42,
    n_jobs=-1,              # 使用所有CPU核心
    max_depth=25,           # 可以探索更深的关系
    min_samples_leaf=5,     # 防止过拟合
    max_features='sqrt'     # 一种常用的特征选择策略
)

# 使用处理好的数据进行训练
final_rf_model.fit(X_train_scaled, Y_train_pca)
print("模型训练完成!")

模型训练完成!


In [14]:
Y_pred = final_rf_model.predict(X_test_scaled)

# 使用处理后的Y_test_pca来计算分数
R2_final = r2_score(Y_test_pca, Y_pred)
RMSE_final = np.sqrt(mean_squared_error(Y_test_pca, Y_pred))

print(f"\n--- ✨ 最终模型评估结果 ✨ ---")
print(f"R² 分数: {R2_final:.4f}")
print(f"均方根误差 (RMSE): {RMSE_final:.4f}")


--- ✨ 最终模型评估结果 ✨ ---
R² 分数: -31.7461
均方根误差 (RMSE): 563.4269


In [ ]:
Y_new_df = pd.DataFrame(Y_new)
print(f"净化前 Y 的形状: {Y_new_df.shape}")

# .dropna(axis=1, how='all') 会删除那些“所有”值为NaN的列
Y_new_cleaned_df = Y_new_df.dropna(axis=1, how='all')

# 检查删除了多少列
n_dropped = Y_new_df.shape[1] - Y_new_cleaned_df.shape[1]
print(f"检测到并删除了 {n_dropped} 个完全无效的特征列。")
print(f"净化后 Y 的形状: {Y_new_cleaned_df.shape}")

# 将清理干净的DataFrame转回Numpy数组
Y_new_cleaned = Y_new_cleaned_df.to_numpy()

净化前 Y 的形状: (1794, 54)
检测到并删除了 1 个完全无效的特征列。
净化后 Y 的形状: (1794, 53)


In [ ]:
import numpy as np
import pandas as pd

# --- 假设您的 X_new 和 Y_new_cleaned 已经在这里准备好了 ---
# X_new 的形状是 (1794, 160)
# Y_new_cleaned 的形状是 (1794, 53)

print("\n--- 开始进行地毯式数据排查，寻找极端异常值 ---")

# =============================================================================
# 步骤 1: 排查输入数据 X_new
# =============================================================================
X_new_df = pd.DataFrame(X_new)

# .describe() 会计算每一列的基本统计数据
# 我们用 .T 来转置表格，让每一行代表一个特征，更方便查看
print("\n--- 输入数据 X_new (频域特征) 的统计描述 ---")
X_stats = X_new_df.describe().T

# 筛选出可能存在问题的特征
# 比如，如果一个特征的标准差(std)为0，说明它是一个常数，没有信息量
# 或者，如果最大值(max)比75%分位数(75%)大几个数量级，说明有极端大值
potential_issues_X = X_stats[
    (X_stats['std'] == 0) |
    (X_stats['max'] > X_stats['75%'] * 1000) |
    (X_stats['min'] < X_stats['25%'] * -1000)
]

print("X_new 的总体统计信息:")
print(X_stats)

if not potential_issues_X.empty:
    print("\n\n⚠️  警告: 在 X_new 中发现以下可能存在问题的特征! ⚠️")
    print(potential_issues_X)
else:
    print("\n✅  在 X_new 中未检测到明显的极端异常值。")


# =============================================================================
# 步骤 2: 排查目标数据 Y_new_cleaned
# =============================================================================
Y_new_cleaned_df = pd.DataFrame(Y_new_cleaned)

print("\n\n--- 目标数据 Y_new_cleaned (声音特征) 的统计描述 ---")
Y_stats = Y_new_cleaned_df.describe().T

potential_issues_Y = Y_stats[
    (Y_stats['std'] < 1e-9) | # 标准差极小也可能是问题
    (Y_stats['max'] > Y_stats['75%'] * 1000) |
    (Y_stats['min'] < Y_stats['25%'] * -1000)
]

print("Y_new_cleaned 的总体统计信息:")
print(Y_stats)

if not potential_issues_Y.empty:
    print("\n\n⚠️  警告: 在 Y_new_cleaned 中发现以下可能存在问题的特征! ⚠️")
    print(potential_issues_Y)
else:
    print("\n✅  在 Y_new_cleaned 中未检测到明显的极端异常值。")


--- 开始进行地毯式数据排查，寻找极端异常值 ---

--- 输入数据 X_new (频域特征) 的统计描述 ---
X_new 的总体统计信息:
      count       mean         std       min       25%       50%        75%  \
0    1794.0  27.290155  249.036224  0.014948  0.851436  2.199614   6.371983   
1    1794.0   1.289290    3.243106  0.006319  0.307415  0.630476   1.350802   
2    1794.0   0.605428    0.836866  0.004161  0.169716  0.353031   0.750296   
3    1794.0   1.047234    1.431301  0.004722  0.267107  0.582395   1.298471   
4    1794.0   0.450130    0.670071  0.001968  0.122812  0.253539   0.513840   
..      ...        ...         ...       ...       ...       ...        ...   
155  1794.0  34.369045  135.580307  0.001199  2.163420  6.406597  16.866975   
156  1794.0   3.112349    4.347631  0.000518  0.671451  1.847828   4.004209   
157  1794.0   1.379851    1.493446  0.000404  0.425604  1.012424   1.851448   
158  1794.0   3.113666    3.823667  0.000670  0.780597  1.698474   4.244743   
159  1794.0   2.180722    4.405178  0.000611  0.275694

In [ ]:



# =============================================================================
# 步骤 2: 使用净化后的数据，以一个更简单的模型进行验证
# =============================================================================

# 1. 划分数据集
X_train, X_test, Y_train, Y_test = train_test_split(
    X_new, Y_new_cleaned, test_size=0.2, random_state=42 # 使用 Y_new_cleaned
)

# 2. 对 Y 进行同样的“填充-标准化-PCA”流程
imputer_y = SimpleImputer(strategy='mean')
scaler_y = StandardScaler()
pca_y = PCA(n_components=0.95)

Y_train_imputed = imputer_y.fit_transform(Y_train)
Y_train_scaled = scaler_y.fit_transform(Y_train_imputed)
Y_train_pca = pca_y.fit_transform(Y_train_scaled)

Y_test_imputed = imputer_y.transform(Y_test)
Y_test_scaled = scaler_y.transform(Y_test_imputed)
Y_test_pca = pca_y.transform(Y_test_imputed)
print(f"净化后的 Y 已被提纯至 {pca_y.n_components_} 个主成分。")


# 3. 对 X 进行标准化
scaler_x = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)


# 4. 使用一个更简单、更稳健的随机森林模型进行验证
#    我们大大降低了模型的复杂度，以避免其学习到“伪规律”
print("\n正在使用一个更稳健的简化模型进行训练...")
robust_rf_model = RandomForestRegressor(
    n_estimators=100,       # 树的数量减少
    random_state=42,
    n_jobs=-1,
    max_depth=10,           # 关键：大大降低树的深度！
    min_samples_leaf=10,    # 关键：要求每个叶节点有更多样本
)

robust_rf_model.fit(X_train_scaled, Y_train_pca)
print("模型训练完成!")


# 5. 评估
Y_pred = robust_rf_model.predict(X_test_scaled)
R2 = r2_score(Y_test_pca, Y_pred)
RMSE = np.sqrt(mean_squared_error(Y_test_pca, Y_pred))

print(f"\n--- ✨ 数据净化与简化模型验证结果 ✨ ---")
print(f"R² 分数: {R2:.4f}")
print(f"均方根误差 (RMSE): {RMSE:.4f}")

净化前 Y 的形状: (1794, 54)
检测到并删除了 1 个完全无效的特征列。
净化后 Y 的形状: (1794, 53)
净化后的 Y 已被提纯至 25 个主成分。

正在使用一个更稳健的简化模型进行训练...
模型训练完成!

--- ✨ 数据净化与简化模型验证结果 ✨ ---
R² 分数: -31.7470
均方根误差 (RMSE): 563.4248


## ✅ 追加：稳健评估（按被试、无泄漏）
不改变你已有的变量名/结构，在末尾追加一段**分组切分 + 无泄漏按被试标准化**的评估代码（使用 `subject_ids_list`, `X_new`, `Y_new`）。
这段代码不会覆盖你原有流程，只是**另给一个稳健分数**作为对照。

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import pearsonr
import numpy as np
import pandas as pd

def macro_pearsonr(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    T = y_true.shape[1]
    rs = []
    for t in range(T):
        yt = y_true[:, t]
        yp = y_pred[:, t]
        if np.std(yt) < 1e-9 or np.std(yp) < 1e-9:
            rs.append(0.0)
        else:
            r, _ = pearsonr(yt, yp)
            rs.append(r)
    return float(np.mean(rs))

def scale_within_subject_train_stats(X, subjects, tr_idx, te_idx):
    X = np.asarray(X, dtype=float)
    S = np.asarray(subjects)
    Xtr, Xte = X[tr_idx].copy(), X[te_idx].copy()
    stats = {}
    for s in np.unique(S[tr_idx]):
        m = Xtr[S[tr_idx]==s].mean(axis=0)
        sd = Xtr[S[tr_idx]==s].std(axis=0, ddof=0)
        sd[sd<1e-12] = 1.0
        stats[s] = (m, sd)
    # global fallback
    gm = Xtr.mean(axis=0)
    gs = Xtr.std(axis=0, ddof=0)
    gs[gs<1e-12] = 1.0
    # transform train
    for s in np.unique(S[tr_idx]):
        m, sd = stats[s]
        mask = (S[tr_idx]==s)
        Xtr[mask] = (Xtr[mask] - m)/sd
    # transform test
    for s in np.unique(S[te_idx]):
        m, sd = stats.get(s, (gm, gs))
        mask = (S[te_idx]==s)
        Xte[mask] = (Xte[mask] - m)/sd
    return Xtr, Xte

# 1) 基于 subject_ids_list 的组感知划分（与原来 0.2 测试占比保持一致）
X_base = np.asarray(X_new)
Y_base = np.asarray(Y_new if 'Y_new_cleaned' not in globals() else Y_new_cleaned)
S = np.asarray(subject_ids_list)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, te_idx = next(gss.split(X_base, Y_base, groups=S))

# 2) 按被试、仅用训练段统计量做标准化（无泄漏）
X_train_ps, X_test_ps = scale_within_subject_train_stats(X_base, S, tr_idx, te_idx)
Y_train_ps, Y_test_ps = Y_base[tr_idx], Y_base[te_idx]

# 3) 复用你 Y 的流程：Imputer -> StandardScaler -> PCA（只在训练集拟合）
imputer_y_rb = SimpleImputer(strategy='mean')
scaler_y_rb = StandardScaler()
pca_y_rb = PCA(n_components=0.95)
Ytr_imp = imputer_y_rb.fit_transform(Y_train_ps)
Ytr_sc = scaler_y_rb.fit_transform(Ytr_imp)
Ytr_pca = pca_y_rb.fit_transform(Ytr_sc)
Yte_imp = imputer_y_rb.transform(Y_test_ps)
Yte_sc = scaler_y_rb.transform(Yte_imp)
Yte_pca = pca_y_rb.transform(Yte_sc)

# 4) 复用你常用的随机森林设置，得到“稳健分数”
rf_rb = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
    max_depth=25,
    min_samples_leaf=5,
    max_features='sqrt'
)
rf_rb.fit(X_train_ps, Ytr_pca)
Yhat_pca = rf_rb.predict(X_test_ps)

R2_rb = r2_score(Yte_pca, Yhat_pca)
MAE_rb = mean_absolute_error(Yte_pca, Yhat_pca)
Pr_rb = macro_pearsonr(Yte_pca, Yhat_pca)

print('\n=== 稳健评估（GroupShuffleSplit, 无泄漏按被试标准化） ===')
print(f'R²: {R2_rb:.4f} | MAE: {MAE_rb:.4f} | Pearson r (macro): {Pr_rb:.4f}')
print(f'训练样本: {len(tr_idx)} | 测试样本: {len(te_idx)} | 测试被试数: {len(np.unique(S[te_idx]))}')
